In [22]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from dateparser import parse
import datetime
from pathlib import Path
from dotenv import load_dotenv
from loguru import logger
import os

In [26]:
## This notebook searches for overlapping Sentinel-1 (EW GRDM) and Sentinel-2 (L1C, cloud coverage 0-30%) data.
### The search is implemented using the 'sentinelsat' package, which is a powerful search API for the Copernicus Scihub database.

### Two requirement to run this script:
### 1) have a user account on Copernicus Open Access Hub: https://scihub.copernicus.eu/dhus/#/home
### 2) create a file called .env in the directory 'S1_sea_ice_classification'. In this file, save your username and password for SciHub like this:
### DHUS_USER=<scihub_username>
### DHUS_PASSWORD=<scihub_password>


#### Load environment variables containing username and password for Copernicus Scihub

In [23]:
load_dotenv()
    
try:
    os.environ["DHUS_USER"]
except:
    logger.error("The environment variable 'DHUS_USER' is not set! Exiting...")
    raise KeyError("The environment variable 'DHUS_USER' is not set!")

try:
    os.environ["DHUS_PASSWORD"]
except:
    logger.error("The environment variable 'DHUS_PASSWORD' is not set! Exiting...")
    raise KeyError("The environment variable 'DHUS_PASSWORD' is not set!")


In [24]:
print(os.environ["DHUS_USER"])

cta014


In [25]:
# establish connection to Copernicus Scihub
api = SentinelAPI(os.environ["DHUS_USER"], os.environ["DHUS_PASSWORD"])

# search over Fram Strait for S1 imagery (from 1st Dec 2020 to 1st June 2021)
# use custom geojson files containing search polygon
area = './rois/Fram_Strait.geojson'
polygon_path = Path(area).expanduser().absolute()

# search from 1st March 2021 to 1st May 2021
starttime = "2020-03-01"
endtime = "2020-05-01"
starttime = parse(starttime, settings={"DATE_ORDER": "YMD"})
endtime = parse(endtime, settings={"DATE_ORDER": "YMD"})

# query for specific product
s1_products = api.query(
                geojson_to_wkt(read_geojson(polygon_path)),
                date=(starttime, endtime),
                platformname="Sentinel-1",
                producttype="GRD",
                )

s1_list_with_overlap = []
s1_s2_overlap = {}

# iterate over query results to check for overlapping S2
for product in s1_products:

    # retrieve some properties of the S1 product
    s1_footprint = s1_products[product]['footprint']
    s1_identifier = s1_products[product]['identifier']
    s1_timestamp = s1_products[product]['beginposition']

    # look for S2 imagery taken 2 hours before or after S1 acquisition
    timedelta = datetime.timedelta(hours = 2) 
    s2_starttime = s1_timestamp - timedelta
    s2_endtime = s1_timestamp + timedelta

    # query for overlapping Sentinel-2 optical images
    s2_products = api.query(area = s1_footprint,
                            date = (s2_starttime, s2_endtime),
                            platformname="Sentinel-2",
                            processinglevel="Level-1C",
                            cloudcoverpercentage=(0, 30)
                            )
    
    # if overlapping S2 scenes found, add the S1 id and corresponding S2 query results in dictionary
    if not len(s2_products) == 0:
        s1_list_with_overlap.append(product)
        if len(s2_products) == 1:
            for s2product in s2_products:
                s2_identifier = s2_products[s2product]['identifier']
                s1_s2_overlap[s1_identifier] = s2_identifier
        elif len(s2_products) > 1:
            s2_id_list = []
            for s2product in s2_products:
                s2_identifier = s2_products[s2product]['identifier']
                s2_id_list.append(s2_identifier)
            s1_s2_overlap[s1_identifier] = s2_id_list
            
# this list is contains the S1 scenes that have overlapping S2 imagery
print('Found', len(s1_list_with_overlap), 'S1 scenes with overlapping S2 imagery.')
print(s1_list_with_overlap)

# make copy of the original query results
s1_products_with_overlap = s1_products.copy()

# loop over all original query results and only keep the ones that have overlapping S2
for product in s1_products:
    if not product in s1_list_with_overlap:
        del s1_products_with_overlap[product]
        
# 's1_products_with_overlap' can be direclty entered into the api.download_all() function. This will then download all S1 images that have overlapping S2 imagery

Querying products:  23%|##3       | 100/426 [00:00<?, ?product/s]

Querying products:  96%|#########6| 100/104 [00:00<?, ?product/s]

Found 15 S1 scenes with overlapping S2 imagery.
['7a9d39fe-7343-43d5-bbfb-2c1c23b873f2', 'e2e44b29-43c5-48f8-b0b2-e9bd150dc542', '73ed5fe4-eada-4ed8-aa9a-83d9f2218777', '11693167-8de4-4529-a779-a00290346b19', '78f1e137-189a-4743-a31a-52537468e780', '6fde56ac-b85b-444e-be41-194d0a686b88', '88c9e908-8c08-4e6a-92fd-e7ecb6e8d060', '81c26764-5ac0-42b7-9917-3d647dde6ead', '42f92139-78a1-4971-a9ed-ec448cf8ecfa', '9c620dee-4040-4b56-9ec4-203e4bfcd3f8', '858d48a5-afe4-4f6c-93f3-bef2bf0116b2', '26fa1a72-8ad4-4eb8-9852-d1adce9782b0', 'a5d38ae2-bbc4-4548-a64a-4fc55dc9bf86', 'fb0d3244-469e-4ff6-9bbe-558bd0b14844', 'f05595d8-7223-4120-8578-7e05348f5245']


In [19]:
print(len(s1_products_with_overlap))

2


In [20]:
print(s1_products_with_overlap.keys())

odict_keys(['b1147e36-9e0f-4ff5-93a0-f5e2d00da8ab', 'c31e230a-524d-4ab0-a442-457ffc65a93b'])


In [21]:
print(s1_s2_overlap.keys())

dict_keys(['S1B_EW_GRDM_1SDH_20210331T174023_20210331T174045_026262_032261_26FA', 'S1A_EW_GRDM_1SDH_20210316T180541_20210316T180652_037027_045B91_98E3'])


In [14]:
print(s1_s2_overlap['S1B_EW_GRDM_1SDH_20210331T174023_20210331T174045_026262_032261_26FA'])

['S2B_MSIL1C_20210331T154809_N0300_R054_T26XMN_20210331T174855', 'S2B_MSIL1C_20210331T154809_N0300_R054_T26XMM_20210331T174855']
